# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,15.67,76,41,9.45,SH,1707070503
1,1,chul'man,56.8479,124.9104,-39.33,100,77,0.47,RU,1707070504
2,2,adamstown,-25.0660,-130.1015,24.85,79,99,3.46,PN,1707070504
3,3,mead valley,33.8334,-117.2962,14.05,72,20,3.60,US,1707070505
4,4,tiksi,71.6872,128.8694,-33.61,100,95,0.39,RU,1707070505


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = 0.5
    )
    
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df= city_data_df.loc[(city_data_df['Max Temp']>=21.03) & (city_data_df['Max Temp']<=26.27)]
ideal_humidity_df= ideal_temp_df.loc[(ideal_temp_df['Humidity']>=55) & (ideal_temp_df['Humidity']<=100)]
idea_weather_df = ideal_humidity_df.loc[(ideal_humidity_df['Wind Speed']>=0.35) & (ideal_humidity_df['Wind Speed']<=3.69)]


# Drop any rows with null values
ideal_temp_df.dropna(how='any', axis=0)

# Display sample data
idea_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,adamstown,-25.0660,-130.1015,24.85,79,99,3.46,PN,1707070504
6,6,mount isa,-20.7333,139.5000,25.87,89,96,2.57,AU,1707070505
19,19,bredasdorp,-34.5322,20.0403,23.78,77,6,2.97,ZA,1707070510
40,40,eenhana,-17.4667,16.3333,23.55,72,100,1.19,NaN,1707070519
47,47,nova sintra,14.8667,-24.7167,21.62,67,0,3.60,CV,1707070522
...,...,...,...,...,...,...,...,...,...,...
530,530,byron bay,-28.6500,153.6167,23.17,72,80,3.58,AU,1707070629
540,540,kendari,-3.9450,122.4989,24.94,93,100,1.48,ID,1707070631
542,542,kalemie,-5.9475,29.1947,23.15,82,99,1.06,CD,1707070632
547,547,simpang,-1.2667,104.0833,24.24,94,100,0.60,ID,1707070633


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idea_weather_df.drop(columns=['City_ID','Cloudiness','Wind Speed','Max Temp','Date']) 
hotel_df=hotel_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,'Hotel Name','')

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,adamstown,PN,-25.0660,-130.1015,79,
6,mount isa,AU,-20.7333,139.5000,89,
19,bredasdorp,ZA,-34.5322,20.0403,77,
40,eenhana,NaN,-17.4667,16.3333,72,
47,nova sintra,CV,14.8667,-24.7167,67,
...,...,...,...,...,...,...
530,byron bay,AU,-28.6500,153.6167,72,
540,kendari,ID,-3.9450,122.4989,93,
542,kalemie,CD,-5.9475,29.1947,82,
547,simpang,ID,-1.2667,104.0833,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.at[index,'Lat']
    longitude = hotel_df.at[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = f'{base_url}?categories=accommodation.hotel&filter={params["filter"]}&bias={params["bias"]}&apiKey={geoapify_key}'
    
    # Convert the API response to JSON format
    name_address = requests.get(name_address).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
bredasdorp - nearest hotel: Victoria Hotel
eenhana - nearest hotel: Monte Carlo Guesthouse
nova sintra - nearest hotel: Residência Ka Dencho
hermanus - nearest hotel: Aloe guest house
port alfred - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
inhambane - nearest hotel: Hotel de Inhambane
wauchope - nearest hotel: No hotel found
conceicao do rio verde - nearest hotel: Pousada Fazenda Velha
cabo san lucas - nearest hotel: Comfort Rooms
kone - nearest hotel: Pacifik Appartels
huarmey - nearest hotel: Hostal Santa Rosa
amahai - nearest hotel: Hotel Irene
mangai - nearest hotel: No hotel found
pasuquin - nearest hotel: Villa del Marj
aripuana - nearest hotel: No hotel found
manakara - nearest hotel: Hotel H1
mananara - nearest hotel: Chez Roger
walvis bay - nearest hotel: Atlantic
kingscliff - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
2,adamstown,PN,-25.0660,-130.1015,79,No hotel found
6,mount isa,AU,-20.7333,139.5000,89,Ibis Styles
19,bredasdorp,ZA,-34.5322,20.0403,77,Victoria Hotel
40,eenhana,NaN,-17.4667,16.3333,72,Monte Carlo Guesthouse
47,nova sintra,CV,14.8667,-24.7167,67,Residência Ka Dencho
...,...,...,...,...,...,...
530,byron bay,AU,-28.6500,153.6167,72,Lord Byron Resort
540,kendari,ID,-3.9450,122.4989,93,City Hotel Kendari
542,kalemie,CD,-5.9475,29.1947,82,Hotel du Lac
547,simpang,ID,-1.2667,104.0833,94,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    color = 'City',
    hover_cols=['Hotel Name','Country'],
    alpha = 0.5,
    size = 'Humidity'
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)